### PIECES LEFT TO DO:

* __Make sure you also are grabbing any file that isn't an MP3__
    * __This may mean grabbing all files and putting them in a table first, then going in and checking their tags__
    * __Or pull everything in, but have new columns for dirpath, dirname, filename so that you can grab all rows with the same dirpath as opposed to just grabbing things that have `.mp3` in the full dirpath__

* __Go through `deds` and build table of correct data for it__

    * __Bonus points if you can find an API or something to hit__

* __Perform the actual file move once everything is up to date__



-----------------



This project will let me see if I can:

* Read id3 tags in Python

* Create a new directory with music files organized by [main]Artist/Album/Song
   
  * as opposed to [Song] Artists/Album/Single Song
    
I am so tired of Apple's music file directory schema. It makes it impossible to listen to a full album from a place like VLC player because each song that has a feature or colab is given its own directory tree instead of just being in the album folder. It pisses me off to no end and I am now going to try to write a script to read everything, organize it, and rewrite the directory.

There is a good chance that this will be harder than I want it to be. Meh. Should be fun!

In [77]:
## https://eyed3.readthedocs.io/en/latest/
from os import listdir
from os.path import isfile, join
from os import walk
import os
import shutil


import eyed3 as id3
import pandas as pd

pd.set_option('display.width', 1000)



dests = pd.DataFrame(columns=['file_origin', 'Artist', 'Album', 'New_Loc'])

## Will want to either loop through all files in a directory tree or generate a list of files in a target directory
### Something like a list of every file, go through each one, grab id3 info
### Then build a theoretical directory tree into a table [?]
### Run a file writer that iterates through each song in the table and creates the appropriate directories in a secondary location

### IF COMPILATION ALBUM/SOUNDTRACK: STORE WITHOUT ARTIST FOLDER, i.e City Lounge 1 will be a folder with songs at the artist level


# mypath = "L:\DeathStar\Music\Music"
# newpath = "D:\Music"

mypath = "/home/james/Music"
newpath = "/home/james/Music/Cleaned"

#onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
#print(listdir(mypath))

files = []

####### THIS IS WHERE YOU WOULD BE MAKING THE ADDITION TO CATCH THUMBNAILS, ETC. #########


for (dirpath, dirnames, filenames) in walk(mypath):
    if filenames:
        for i in filenames:
            fp = os.path.join(dirpath, i)
            files += [fp]
print("Files Read!\nFound {} files".format(len(files)))





deds = []
cnt = 0
for song in files:
    if song.endswith(".mp3"):        
        #row = [song]
        #print(song)
        try:
            test_file = id3.load(song)
        except Exception as err:
            print("{}\nError: {}".format(song, err))
        else:
            try:
                tit = test_file.tag.title
                song_file = tit + ".mp3"
                art = test_file.tag.artist
                alb = test_file.tag.album
                npth = os.path.join(newpath,art,alb,song_file)
            except Exception as err:
                print("Could not get album or artist info for {}\nError: {}".format(song,err))
                deds += [song]
                continue
            else:
                #print("Song: {}".format(tit))
                #print("Artist: {}".format(art))
                #print("Album: {}".format(alb))
                
                if art and alb:
                 #   print("New Path Would Be:\n{}".format(npth))
                    row = pd.DataFrame(data={'file_origin': [song], 'Artist': [art], 'Album': [alb], 'New_Loc': [npth]})
                    dests = dests.append(row)
                else:
                  #  print("Could not get id3 tag info for {}".format(song))
                    deds += [song]

                #shutil.copyfile2(song, os.path.join(art,alb))
            cnt += 1
            print("Finished with {} of {}".format(cnt, len(files)))
            print('-------------------')

## Save dests as a csv so that you don't have to rerun this on the giant sample of ~18k songs?
            
            
            

#################################################
#                                               #
#                    NOTES                      #
#                                               #
#################################################


# Need to separate out artist folders from compilation album folders
## My method is returning some of the same garbage for albums like city lounge
### Soundtracks will also be fucked
    
# There will just have to be some manual work, darn



# Before copying, test to see if the full fiepath exists before copying
## might help roust out duplicates

# OR build in logic:
#  -create csv/dataframe of all suggested/scraped id3 tags
#  -try to see if there are albums with more than one artist
    # something like for each album, count number of unique artsits
        # This will also require a bit of oversight as some albums will be tricky
#  -if more than one
    # find out if it is a compilation album (manual maybe)
    # if it's just a random artist on the album and have the directory be the majority artist on the album
        # This may not even be neccessary but it's not a bad idea



#################################################


## Load the song file
#test_file = id3.load("/home/james/Music/Ghostface Killah & Amy Winehouse/More Fish/14 You Know I'm No Good.mp3")

## Primary Folder: Artist
#print(test_file.tag.artist)

## Secondary Folder: Album
#print(test_file.tag.album)






FrameHeader: Illegal Frame ID: b'CM1\x00'


Files Read!
Found 328 files
Finished with 1 of 328
-------------------
Finished with 2 of 328
-------------------
Finished with 3 of 328
-------------------
Finished with 4 of 328
-------------------
Finished with 5 of 328
-------------------
Finished with 6 of 328
-------------------
Finished with 7 of 328
-------------------
Finished with 8 of 328
-------------------
Finished with 9 of 328
-------------------
Finished with 10 of 328
-------------------
Finished with 11 of 328
-------------------
Finished with 12 of 328
-------------------
Finished with 13 of 328
-------------------
Finished with 14 of 328
-------------------
Finished with 15 of 328
-------------------
Finished with 16 of 328
-------------------
Finished with 17 of 328
-------------------
Finished with 18 of 328
-------------------
Finished with 19 of 328
-------------------
Finished with 20 of 328
-------------------
Finished with 21 of 328
-------------------
Finished with 22 of 328
-------------------
Finished with

Lame tag CRC check failed
Lame tag CRC check failed


Finished with 28 of 328
-------------------
Finished with 29 of 328
-------------------
Finished with 30 of 328
-------------------
Finished with 31 of 328
-------------------
Finished with 32 of 328
-------------------
Finished with 33 of 328
-------------------
Finished with 34 of 328
-------------------
Finished with 35 of 328
-------------------
Finished with 36 of 328
-------------------
Finished with 37 of 328
-------------------
Finished with 38 of 328
-------------------
Finished with 39 of 328
-------------------
Finished with 40 of 328
-------------------
Finished with 41 of 328
-------------------
Finished with 42 of 328
-------------------
Finished with 43 of 328
-------------------
Finished with 44 of 328
-------------------
Finished with 45 of 328
-------------------
Finished with 46 of 328
-------------------
Finished with 47 of 328
-------------------
Finished with 48 of 328
-------------------
Finished with 49 of 328
-------------------
Finished with 50 of 328
--------

Lame tag CRC check failed
Lame tag CRC check failed
Lame tag CRC check failed
Lame tag CRC check failed
Lame tag CRC check failed
Lame tag CRC check failed
Lame tag CRC check failed
Lame tag CRC check failed
Lame tag CRC check failed
Lame tag CRC check failed
Lame tag CRC check failed
Lame tag CRC check failed
Lame tag CRC check failed
Lame tag CRC check failed


...

Finished with 171 of 328
-------------------
Finished with 172 of 328
-------------------
Finished with 173 of 328
-------------------
Finished with 174 of 328
-------------------
Finished with 175 of 328
-------------------
Finished with 176 of 328
-------------------
Finished with 177 of 328
-------------------
Finished with 178 of 328
-------------------
Finished with 179 of 328
-------------------
Finished with 180 of 328
-------------------
Finished with 181 of 328
-------------------
Finished with 182 of 328
-------------------
Finished with 183 of 328
-------------------
Finished with 184 of 328
-------------------
Finished with 185 of 328
-------------------
Finished with 186 of 328
-------------------
Finished with 187 of 328
-------------------
Finished with 188 of 328
-------------------
Finished with 189 of 328
-------------------
Finished with 190 of 328
-------------------
Finished with 191 of 328
-------------------
Finished with 192 of 328
-------------------
Finished w

GEOB frame does not contain a mime type
GEOB frame does not contain a valid mime type
GEOB frame does not contain a mime type
GEOB frame does not contain a valid mime type
GEOB frame does not contain a mime type
GEOB frame does not contain a valid mime type
GEOB frame does not contain a mime type
GEOB frame does not contain a valid mime type


Finished with 194 of 328
-------------------
Finished with 195 of 328
-------------------
Finished with 196 of 328
-------------------
Finished with 197 of 328
-------------------
Finished with 198 of 328
-------------------
Finished with 199 of 328
-------------------
Finished with 200 of 328
-------------------
Finished with 201 of 328
-------------------
Finished with 202 of 328
-------------------
Finished with 203 of 328
-------------------
Finished with 204 of 328
-------------------
Finished with 205 of 328
-------------------
Finished with 206 of 328
-------------------
Finished with 207 of 328
-------------------
Finished with 208 of 328
-------------------
Finished with 209 of 328
-------------------
Finished with 210 of 328
-------------------
Finished with 211 of 328
-------------------
Finished with 212 of 328
-------------------
Finished with 213 of 328
-------------------
Finished with 214 of 328
-------------------
Finished with 215 of 328
-------------------
Finished w

GEOB frame does not contain a mime type
GEOB frame does not contain a valid mime type
GEOB frame does not contain a mime type
GEOB frame does not contain a valid mime type
GEOB frame does not contain a mime type
GEOB frame does not contain a valid mime type
GEOB frame does not contain a mime type
GEOB frame does not contain a valid mime type
GEOB frame does not contain a mime type
GEOB frame does not contain a valid mime type
GEOB frame does not contain a mime type
GEOB frame does not contain a valid mime type
GEOB frame does not contain a mime type
GEOB frame does not contain a valid mime type
GEOB frame does not contain a mime type
GEOB frame does not contain a valid mime type
GEOB frame does not contain a mime type
GEOB frame does not contain a valid mime type
GEOB frame does not contain a mime type
GEOB frame does not contain a valid mime type
GEOB frame does not contain a mime type
GEOB frame does not contain a valid mime type
GEOB frame does not contain a mime type
GEOB frame doe

Finished with 220 of 328
-------------------
Finished with 221 of 328
-------------------
Finished with 222 of 328
-------------------
Finished with 223 of 328
-------------------
Finished with 224 of 328
-------------------
Finished with 225 of 328
-------------------
Finished with 226 of 328
-------------------
Finished with 227 of 328
-------------------
Finished with 228 of 328
-------------------
Finished with 229 of 328
-------------------
Finished with 230 of 328
-------------------
Finished with 231 of 328
-------------------
Finished with 232 of 328
-------------------
Finished with 233 of 328
-------------------
Finished with 234 of 328
-------------------
Finished with 235 of 328
-------------------
Finished with 236 of 328
-------------------
Finished with 237 of 328
-------------------
Finished with 238 of 328
-------------------
Finished with 239 of 328
-------------------
Finished with 240 of 328
-------------------
Finished with 241 of 328
-------------------
Finished w

In [73]:
from numpy import std
import re

# To make sure that we capture all characters in dir path strings from the dataframe
# Otherwise when it's converting with to_string(), it cuts off after X characters which is 
# converting the displayed output to a string but not the value itself. Stupid.
pd.options.display.max_colwidth = 1000


for i in list(dests.Album.unique()):
    print(i)    
    albs = dests[dests.Album == i]
    arts = albs.Artist.unique()
    if len(arts) > 1:
        print("More than one artist")
        ch_tbl = pd.DataFrame({"Artist": [], "Count" : [], "PCT": []})
        
        for t in arts:
            ch_art = t
            ch_nums = len(albs[albs.Artist == t])
            ch_pct = round((ch_nums/len(albs)) * 100, 1)
            ch_row = pd.DataFrame({"Artist": [ch_art], "Count" : [ch_nums], "PCT": [ch_pct]})
            ch_tbl = ch_tbl.append(ch_row)
        
        ## Catch Compilation Albums based on standard deviation
        
        if std(ch_tbl.PCT) < 3:     # Will want to figure out if 3 is the right thereshold.
            print("Compilation Album!")
            for t in arts:   
                old_loc = str(dests.New_Loc[(dests.Album == i) & (dests.Artist == t)].to_string(index=False))
                #print(t)
                #print("===============================")
                for line in [old_loc]:
                    
                    #print(str(line))
                    #print("++++++++++++++++\nNew Loc:")
                    new_loc = re.sub(pattern=str(t), repl="", string=line)
                    new_loc = str(re.sub(pattern="//", repl="/", string=new_loc))
                    #print(new_loc)
                    if line == new_loc:
                        print("Something ain't right here!")
                    else:
                        dests.New_Loc[(dests.New_Loc == line)] = new_loc
               # print("------------\n------------\n------------")
               # print(dests[(dests.Album == i) & (dests.Artist == t)])
               # print("------------\n------------\n------------")
        
        ## Replace folder naming if there are a few non-album artists in the album. ID3 tag stays the same
        
        else:
            rep = ch_tbl.sort_values(by=['PCT'],ascending=[False])
            rep = rep.Artist.iloc[0]         
            for t in arts:
                if t == rep:
                    continue
                else:
                    old_loc = str(dests.New_Loc[(dests.Album == i) & (dests.Artist == t)].to_string(index=False))
                    #print(t)
                    #print("===============================")
                    for line in [old_loc]:
                     #   print(str(line))
                      #  print("++++++++++++++++\nNew Loc:")
                        new_loc = re.sub(pattern=str(t), repl=rep, string=line)
                        #new_loc = str(re.sub(pattern="//", repl="/", string=new_loc))
                       # print(new_loc)
                        if line == new_loc:
                            print("Something ain't right here!")
                        else:
                            dests.New_Loc[(dests.New_Loc == line)] = new_loc
                    #print("------------\n------------\n------------")
                    #print(dests[(dests.Album == i) & (dests.Artist == t)])
                    #print("------------\n------------\n------------")        
    else:
        print("This looks just like an album!")
    print("-------------------------------------------")
    
    #print(dests.loc[i])
    


More Fish
This looks just like an album!
-------------------------------------------
Supreme Clientele
This looks just like an album!
-------------------------------------------
The Wallabee Champ
This looks just like an album!
-------------------------------------------
Face Of The Ghost  Bootleg
More than one artist
Hundred Reasons
/home/james/Music/Cleaned/Hundred Reasons/Face Of The Ghost  Bootleg/Love Stories (Feat Otherized F).mp3
++++++++++++++++
New Loc:
/home/james/Music/Cleaned/Ghostface Killah/Face Of The Ghost  Bootleg/Love Stories (Feat Otherized F).mp3
------------
------------
------------
                                                                                        file_origin           Artist                       Album                                                                                                    New_Loc
0  /home/james/Music/Ghostface Killah/Face Of The Ghost  Bootleg/Love Stories (Feat Otherized F.mp3  Hundred Reasons  Face Of The Ghost 

------------
                                                                                                       file_origin          Artist                 Album                                                          New_Loc
0  /home/james/Music/City Lounge  New York Edition Vol. 1.www.lokotorrents.com/8. Blue Horizon- Suntheca Prod..mp3  Suntheca Prod.  City Lounge New York  /home/james/Music/Cleaned/City Lounge New York/Blue Horizon.mp3
------------
------------
------------
The Mask 
/home/james/Music/Cleaned/The Mask /City Lounge New York/Afreakatum (Suntheca Lounge Mix).mp3
++++++++++++++++
New Loc:
/home/james/Music/Cleaned/City Lounge New York/Afreakatum (Suntheca Lounge Mix).mp3
------------
------------
------------
                                                                                                                       file_origin     Artist                 Album                                                                              New_Loc
0  /home/j

Silent Breeze
/home/james/Music/Cleaned/Silent Breeze/City Lounge San Francisco 2011/Rain Clouds.mp3
++++++++++++++++
New Loc:
/home/james/Music/Cleaned/City Lounge San Francisco 2011/Rain Clouds.mp3
------------
------------
------------
                                                                                                              file_origin         Artist                           Album                                                                   New_Loc
0  /home/james/Music/City Lounge San Francisco Edition .2011[www.lokotorrents.com][mp3]/1 Silent Breeze - Rain Clouds.mp3  Silent Breeze  City Lounge San Francisco 2011  /home/james/Music/Cleaned/City Lounge San Francisco 2011/Rain Clouds.mp3
------------
------------
------------
Cane Garden Quartet
/home/james/Music/Cleaned/Cane Garden Quartet/City Lounge San Francisco 2011/Chillaxin.mp3
++++++++++++++++
New Loc:
/home/james/Music/Cleaned/City Lounge San Francisco 2011/Chillaxin.mp3
------------
------------
--

0  /home/james/Music/City Lounge 8 - The Cool Tempo New Generation.2011[www.lokotorrents.com][mp3]/CD 4 - Berlin/10. TRICKSKI - At Les (Carl Craig Cover).mp3  Trickski  City Lounge Vol. 08  /home/james/Music/Cleaned/City Lounge Vol. 08/At Les (Carl Craig Cover).mp3
------------
------------
------------
Soulphiction
/home/james/Music/Cleaned/Soulphiction/City Lounge Vol. 08/Angela.mp3
++++++++++++++++
New Loc:
/home/james/Music/Cleaned/City Lounge Vol. 08/Angela.mp3
------------
------------
------------
                                                                                                                                   file_origin        Artist                Album                                                   New_Loc
0  /home/james/Music/City Lounge 8 - The Cool Tempo New Generation.2011[www.lokotorrents.com][mp3]/CD 4 - Berlin/11. SOULPHICTION - Angela.mp3  Soulphiction  City Lounge Vol. 08  /home/james/Music/Cleaned/City Lounge Vol. 08/Angela.mp3
------------
-----

------------
                                                                                                                                                  file_origin                   Artist                Album                                                       New_Loc
0  /home/james/Music/City Lounge 8 - The Cool Tempo New Generation.2011[www.lokotorrents.com][mp3]/CD 3 - London/15. THE CINEMATIC ORCHESTRA - Child Song.mp3  The Cinematic Orchestra  City Lounge Vol. 08  /home/james/Music/Cleaned/City Lounge Vol. 08/Child Song.mp3
------------
------------
------------
Beat Assailant
/home/james/Music/Cleaned/Beat Assailant/City Lounge Vol. 08/Spy (Feat. Ben L'Oncle Soul).mp3
++++++++++++++++
New Loc:
/home/james/Music/Cleaned/City Lounge Vol. 08/Spy (Feat. Ben L'Oncle Soul).mp3
------------
------------
------------
                                                                                                                                                      file_origin  

In [2]:
#os.path.join(art,alb)
for i in deds:
    ftt = id3.load(i)
    #print(dir(ftt))        
    #print(dir(ftt.tag))
    print("Album: {}".format(ftt.tag.album))
    print("Album Artist: {}".format(ftt.tag.album_artist))
    print("Album Type: {}".format(ftt.tag.album_type))
    print("Artist: {}".format(ftt.tag.artist))
    print("Artist Origin: {}".format(ftt.tag.artist_origin))
    print("Artist URL: {}".format(ftt.tag.artist_url))
    print("Audio File URL: {}".format(ftt.tag.audio_file_url))
    print("Audio Source URL: {}".format(ftt.tag.audio_source_url))
    print("Best Release Date: {}".format(ftt.tag.best_release_date))
    print("BPM: {}".format(ftt.tag.bpm))
    print("CD ID: {}".format(ftt.tag.cd_id))
    print("Chapters: {}".format(ftt.tag.chapters))
    print("Comments: {}".format(ftt.tag.comments))
    print("Commercial URL: {}".format(ftt.tag.commercial_url))
    print("Composer: {}".format(ftt.tag.composer))
    print("Copyright URL: {}".format(ftt.tag.copyright_url))
    print("Disk Num: {}".format(ftt.tag.disc_num))
    print("Encoding Date: {}".format(ftt.tag.encoding_date))
    print("Extended Header: {}".format(ftt.tag.extended_header))
    print("File Info: {}".format(ftt.tag.file_info))
    print("Frame Set: {}".format(ftt.tag.frame_set))
    print("Frameiter: {}".format(ftt.tag.frameiter))
    print("Genre: {}".format(ftt.tag.genre))
    print("Header: {}".format(ftt.tag.header))
    print("Images: {}".format(ftt.tag.images))
    print("Internet Radio URL: {}".format(ftt.tag.internet_radio_url))
    print("isV1: {}".format(ftt.tag.isV1))
    print("isV2: {}".format(ftt.tag.isV2))
    print("Lyrics: {}".format(ftt.tag.lyrics))
    print("Non Standard Genre: {}".format(ftt.tag.non_std_genre))
    print("Objects: {}".format(ftt.tag.objects))
    print("Original Release Date: {}".format(ftt.tag.original_release_date))
    print("Parse: {}".format(ftt.tag.parse))
    print("Payment_url: {}".format(ftt.tag.payment_url))
    print("Play Count: {}".format(ftt.tag.play_count))
    print("Popularities: {}".format(ftt.tag.popularities))
    print("Privates: {}".format(ftt.tag.privates))
    print("Publisher: {}".format(ftt.tag.publisher))
    print("Publisher URL: {}".format(ftt.tag.publisher_url))
    print("Read Only: {}".format(ftt.tag.read_only))
    print("Recording Date: {}".format(ftt.tag.recording_date))
    print("Release Date: {}".format(ftt.tag.release_date))
    print("SetTextFrame: {}".format(ftt.tag.setTextFrame))
    print("Table of Contents: {}".format(ftt.tag.table_of_contents))
    print("Tagging Date: {}".format(ftt.tag.tagging_date))
    print("Terms of Use: {}".format(ftt.tag.terms_of_use))
    print("Title: {}".format(ftt.tag.title))
    print("Track Number: {}".format(ftt.tag.track_num))
    print("Unique File Ids: {}".format(ftt.tag.unique_file_ids))
    print("User Text Frames: {}".format(ftt.tag.user_text_frames))
    print("User Url Frames: {}".format(ftt.tag.user_url_frames))
    print("Version: {}".format(ftt.tag.version))
    #break
    print("================\n===============")
    

FrameHeader: Illegal Frame ID: b'\xffLEN'
/home/james/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:14: DeprecationWarning: Use Tag.getBestDate() instead
  
FrameHeader: Illegal Frame ID: b'\xffLEN'
FrameHeader: Illegal Frame ID: b'\xffLEN'
FrameHeader: Illegal Frame ID: b'\xffLEN'
FrameHeader: Illegal Frame ID: b'\xffLEN'


Album: None
Album Artist: None
Album Type: None
Artist: None
Artist Origin: [None, None, None]
Artist URL: None
Audio File URL: None
Audio Source URL: None
Best Release Date: None
BPM: None
CD ID: b'\x00z\x01\x0e\x00\x12\x01\x00\x00\x00\x00\x00\x00\x12\x02\x00\x00\x00B\xd4\x00\x12\x03\x00\x00\x00\x93S\x00\x12\x04\x00\x00\x00\xddC\x00\x12\x05\x00\x00\x01-\x9e\x00\x12\x06\x00\x00\x01Z\x93\x00\x12\x07\x00\x00\x01\x8e1\x00\x12\x08\x00\x00\x01\xd2e\x00\x12\t\x00\x00\x02!@\x00\x12\n\x00\x00\x02ne\x00\x12\x0b\x00\x00\x02\xaf\x08\x00\x12\x0c\x00\x00\x02\xf8\x1c\x00\x12\r\x00\x00\x03NE\x00\x12\x0e\x00\x00\x03\x95\x96\x00\x12\xaa\x00\x00\x03\xdc\xbf\xff\xff'
Chapters: <eyed3.id3.tag.ChaptersAccessor object at 0x7f202f3be3c8>
Comments: <eyed3.id3.tag.CommentsAccessor object at 0x7f202f3be278>
Commercial URL: None
Composer: None
Copyright URL: None
Disk Num: (None, None)
Encoding Date: None
Extended Header: <eyed3.id3.headers.ExtendedTagHeader object at 0x7f202f3be198>
File Info: <eyed3.id3.tag.Fi